In [1]:
import gym
import gym.wrappers
import numpy as np
import matplotlib.pylab as pl
%matplotlib inline

from tqdm import tqdm_notebook as tqdm

In [2]:
env = gym.make("MountainCar-v0")
if isinstance(env, gym.wrappers.TimeLimit):
    env = env.env
n_actions = env.action_space.n

print("n_actions=%i" % (n_actions))

[2017-04-11 17:49:24,473] Making new env: MountainCar-v0


n_actions=3


In [3]:
from sklearn.neural_network import MLPClassifier
agent = MLPClassifier(hidden_layer_sizes=(20,20),
                      activation='tanh',
                      warm_start=True, #keep progress between .fit(...) calls
                      max_iter=1 #make only 1 iteration on each .fit(...)
                     )
#initialize agent to the dimension of state an amount of actions
agent.fit([env.reset()]*n_actions,range(n_actions));

/home/shmuma/sfw/anaconda/envs/pract_rl/lib/python3.5/site-packages/sklearn/neural_network/multilayer_perceptron.py:563: ConvergenceWarning: Stochastic Optimizer: Maximum iterations reached and the optimization hasn't converged yet.
  % (), ConvergenceWarning)


In [4]:
def generate_session(t_max=10**4):
    states, actions = [], []
    total_reward = 0
    s = env.reset()
    
    for _ in range(t_max):
        probs = agent.predict_proba([s])[0]
        a = np.random.choice(n_actions, p=probs)
        new_s, r, done, _ = env.step(a)
        states.append(s)
        actions.append(a)
        total_reward += r
        s = new_s
        if done:
            break
            
    return states, actions, total_reward

In [5]:
n_samples = 250
percentile = 50

for i in range(10):
    %time sessions = [generate_session() for _ in tqdm(range(n_samples))]
    batch_states, batch_actions, batch_rewards = map(np.array, zip(*sessions))
    
    threshold = np.percentile(batch_rewards, percentile)
    elite_states = batch_states[batch_rewards > threshold]
    elite_actions = batch_actions[batch_rewards > threshold]
    
    if not elite_states.size:
        print("%d: no elite sessions, try again" % i)
        continue
    
    elite_states, elite_actions = map(np.concatenate, [elite_states, elite_actions])
    
    agent.fit(elite_states, elite_actions)
    print("%d: mean reward = %.5f (max: %f)\tthreshold = %.1f" % (
        i, np.mean(batch_rewards), np.max(batch_rewards), threshold))

Widget Javascript not detected.  It may not be installed or enabled properly.


KeyboardInterrupt: 